# Devoir 3 :PRÉTRAITEMENT DE TEXTE MÉDICAL COMPLEXE

In [111]:
import re
from nltk import word_tokenize, sent_tokenize

## text to process

In [112]:
texte_medical = """
Pat. 45ans admis le 12/03/25 pour COVID-19 sévère.
CRP: 125mg/L (N<5) - Fièvre à 39.2°C.
Ttt par corticoïdes IV + azithromycine 500mg/J.
#Attention: allergie à la pénicilline!
"""

In [113]:

abreviations = {
    "BID": "bis in die",
    "IV": "intraveineuse"
}

corrections_medicales = {
    "pat\\.": "patient",
    "Ttt": "traitement",
    "corticoïdes": "glucocorticoides",
    r"\+\+fébrile": "hyperthermique",
    "hyp0glycémie": "hypoglycémie",
    "IV": "intraveineuse"
}

### Nettoyage

In [114]:
import re

def nettoyage(texte):
    # 1. Remplacer les abréviations et erreurs de transcription
    for abbr, remplacement in corrections_medicales.items():
        texte = re.sub(abbr, remplacement, texte, flags=re.IGNORECASE)
    
    # 2. Supprimer les dates (ex: 12/03/25)
    texte = re.sub(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', '', texte)
    
    # 3. Supprimer les valeurs numériques avec unités (ex: 125mg/L)
    texte = re.sub(r'\b\d+(?:\.\d+)?\s*[a-zA-Z/]+(?:/[a-zA-Z]+)?\b', '', texte)
    
    # 4. Supprimer les caractères spéciaux inutiles (#, °, etc.)
    texte = re.sub(r'[#°]', '', texte)
    
    # 5. Supprimer les espaces multiples inutiles
    texte = re.sub(r'\s+', ' ', texte).strip()
    
    return texte




### tokenization

In [115]:
def tokeniser_texte(texte):
    # Séparer les doses (ex: 500mg/J → ['500', 'mg', '/', 'J'])
    texte = re.sub(r'(\d+)(mg|g|ml|L|UI|µg)', r'\1 \2', texte)
    texte = re.sub(r'([a-zA-Z]+)(/)([a-zA-Z]+)', r'\1 / \3', texte)
    # Conserver les concepts composés (COVID-19, etc.)
    tokens = re.findall(r'\b\w+(?:-\w+)*\b|[^\w\s]', texte)
    return tokens

### Normalisation

In [116]:
import re

def normalisation_experte(tokens):
    nouveaux_tokens = []
    
    for token in tokens:
        token_normalise = token
        for motif, remplacement in corrections_medicales.items():
            if re.fullmatch(motif, token):
                token_normalise = remplacement
                break
        nouveaux_tokens.append(token_normalise)
    
    return nouveaux_tokens


### Pipeline


In [117]:
def Pipline(texte):
    texte = nettoyage(texte)
    token = tokeniser_texte(texte)
    normalizedtoken = normalisation_experte(token)
    return normalizedtoken

In [118]:
result = Pipline(texte_medical)
result

['patient',
 'admis',
 'le',
 'pour',
 'COVID-19',
 'sévère',
 '.',
 'CRP',
 ':',
 '(',
 'N',
 '<',
 '5',
 ')',
 '-',
 'Fièvre',
 'à',
 '39',
 '.',
 '2C',
 '.',
 'traitement',
 'par',
 'glucocorticoides',
 'intraveineuse',
 '+',
 'azithromycine',
 '.',
 'Attention',
 ':',
 'allergie',
 'à',
 'la',
 'pénicilline',
 '!']

bonus(deleting the stop-words)

In [119]:
from nltk.corpus import stopwords
def stopwordsdelete(result):
    stopwordslist = set(stopwords.words('french'))
    final_tokens = [token for token in result if token.lower() not in stopwordslist]
    return final_tokens

In [120]:
import nltk
nltk.download('stopwords')




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AMGZA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [121]:
finaltokens = stopwordsdelete(result)
finaltokens

['patient',
 'admis',
 'COVID-19',
 'sévère',
 '.',
 'CRP',
 ':',
 '(',
 '<',
 '5',
 ')',
 '-',
 'Fièvre',
 '39',
 '.',
 '2C',
 '.',
 'traitement',
 'glucocorticoides',
 'intraveineuse',
 '+',
 'azithromycine',
 '.',
 'Attention',
 ':',
 'allergie',
 'pénicilline',
 '!']

# Tester le pipline sur des comptes-rendus hospitaliers

In [126]:
compterendus1 ="""
Pat. 68ans, consulté le 05/04/25 pr dyspnée aiguë.
TDM thoracique montre COVID-19 avec opacités bilatérales.
Ttt par O2 à 4L/min + corticothérapie IV.
#Notes: hyp0glycémie détectée ce matin.

"""


compterendus2 ="""
Pat. 23ans admis le 18/02/25 pour trauma crânien.
CRP: 230mg/L - Température: 38.5°C.
Ttt initial: paracétamol 1g/6h IV + surveillance BID.
#Attention: antécédent allergie pénicilline++.


"""
compterendus3 ="""
Pat. 75ans en USI, 10/03/25, pour pneumopathie sévère.
Fièvre ++fébrile à 40°C, O2 6L/min.
Ttt: azithromycine 500mg/J, suivi glycémie car risque hyp0glycémie.

"""

In [125]:
test = [compterendus1, compterendus2, compterendus3]

for texte in test:
    result = Pipline(texte)
    finaltokens = stopwordsdelete(result)

    print(finaltokens)
    print("\n")
    print("--------------------------------------------------")
    print("\n")

['patient', ',', 'consulté', 'pr', 'dyspnée', 'aiguë', '.', 'TDM', 'thoracique', 'montre', 'COVID', '-', 'opacités', 'bilatérales', '.', 'traitement', 'O2', '+', 'corticothérapie', 'intraveineuse', '.', 'Notes', ':', 'hypoglycémie', 'détectée', 'matin', '.']


--------------------------------------------------


['patient', 'admis', 'trauma', 'crânien', '.', 'CRP', ':', '-', 'Température', ':', '38', '.', '5C', '.', 'traitement', 'initial', ':', 'paracétamol', 'intraveineuse', '+', 'surveillance', 'BID', '.', 'Attention', ':', 'antécédent', 'allergie', 'pénicilline', '+', '+', '.']


--------------------------------------------------


['patient', 'USI', ',', ',', 'pneumopathie', 'sévère', '.', 'Fièvre', 'hyperthermique', '40C', ',', 'O2', '.', 'traitement', ':', 'azithromycine', ',', 'suintraveineusei', 'glycémie', 'car', 'risque', 'hypoglycémie', '.']


--------------------------------------------------


